In [20]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

In [21]:
data=pd.read_csv('full_data.csv')

In [22]:
movies=data.заголовок

In [23]:
top_films = data.groupby('заголовок')[['vote_average']].mean().sort_values('vote_average', ascending=False)
top_films.head(10)

,vote_average
заголовок,
"Танцовщица, Texas Pop. 81 (1998)",10.0
Криминальное чтиво (1994),8.3
Список Шиндлера (1993),8.3
Бойцовский клуб (1999),8.3
Энн из Зеленых Мезонинов (1985),8.2
Пролетая над гнездом кукушки (1975),8.2
Американская история X (1998),8.2
Славные парни (1990),8.2
Однажды в Америке (1984),8.2


In [24]:
# возьмем только фильмы с наивысшей средней оценкой в 5.0
films_with_highest_marks = top_films.iloc[np.where(top_films.vote_average >= 7.5)].index

In [25]:
# достанем по каждому фильму количество рейтингов
title_num_ratings = {}

for title, group in tqdm_notebook(data.groupby('заголовок')):
    title_num_ratings[title] = group.movieId.unique().shape[0]

  0%|          | 0/2536 [00:00<?, ?it/s]

In [26]:
sorted([(title_num_ratings[f], f) for f in films_with_highest_marks], key=lambda x: x[0], reverse=True)[:10]

[(1, ' Танцовщица, Texas Pop. 81 (1998)'),
 (1, ' Криминальное чтиво (1994)'),
 (1, ' Список Шиндлера (1993)'),
 (1, ' Бойцовский клуб (1999)'),
 (1, ' Энн из Зеленых Мезонинов (1985)'),
 (1, ' Пролетая над гнездом кукушки (1975)'),
 (1, ' Американская история X (1998)'),
 (1, ' Славные парни (1990)'),
 (1, ' Однажды в Америке (1984)'),
 (1, ' Форрест Гамп (1994)')]

In [27]:
cast=pd.read_csv('cast_bert.csv')
directors=pd.read_csv('directors_bert.csv')
genres=pd.read_csv('genres_bert.csv')
keywords=pd.read_csv('keywords_bert.csv')

In [28]:
data.columns

Index(['заголовок', 'movieId', 'title', 'id', 'genres', 'cast', 'vote_average',
       'director', 'keywords', 'title_key', 'title_ru', 'directors_x1',
       'directors_x2', 'directors_cluster', 'genre_x1', 'genre_x2', 'genre_x3',
       'genre_cluster', 'cast_x1', 'cast_x2', 'cast_cluster', 'keywords_x1',
       'keywords_x2', 'keywords_cluster'],
      dtype='object')

In [29]:
X=pd.concat([cast, directors, genres, keywords,data.vote_average], axis=1)

In [30]:
X.shape

(2537, 3073)

In [31]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import laplacian_kernel

In [36]:
cosine_sim = manhattan_distances(X, X)

In [37]:
indices = pd.Series(data.index, index = data['заголовок']).drop_duplicates()
indices

заголовок
 История игрушек (1995)          0
 Золотой глаз (1995)             1
 Никсон (1995)                   2
 Остров головорезов (1995)       3
 Казино (1995)                   4
                              ... 
 Инфильтратор (2016)          2532
 Плохие мамочки (2016)        2533
 Площадь (2017)               2534
 Сияние (1997)                2535
 Буйство (2018)               2536
Length: 2537, dtype: int64

In [38]:
def get_recommendation(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    sim_index = [i[0] for i in sim_scores]
    print(data["заголовок"].iloc[sim_index])

In [41]:
get_recommendation(' Сияние (1997)')

1195                 Тупак: Воскрешение (2003)
329           Большое приключение Барни (1998)
1039                          Унесенные (2002)
1312                        Мистер 3000 (2004)
1153                   Открытый полигон (2003)
1012     Мартин Лоуренс Live: Runteldat (2002)
642                  Шанхайский сюрприз (1986)
2259                       Все потеряно (2013)
648                  Школьное изумление (1988)
2084                Счастливые ноги два (2011)
Name: заголовок, dtype: object


Metrics for Co filtering

In [42]:
def check_inputs(func) :
    """
    Decorator function to validate inputs to precision_at_k & recall_at_k
    """
    def checker(df: pd.DataFrame, k: int=3, y_test: str='y_actual', y_pred: str='y_recommended') -> float:
        # check we have a valid entry for k
        if k <= 0:
            raise ValueError(f'Value of k should be greater than 1, read in as: {k}')
        # check y_test & y_pred columns are in df
        if y_test not in df.columns:
            raise ValueError(f'Input dataframe does not have a column named: {y_test}')
        if y_pred not in df.columns:
            raise ValueError(f'Input dataframe does not have a column named: {y_pred}')
        return func(df, k, y_test, y_pred)
    return checker

@check_inputs
def precision_at_k(df: pd.DataFrame, k: int, y_test: str, y_pred: str) -> float:
    """
    Function to compute precision@k for an input boolean dataframe
    
    Inputs:
        df     -> pandas dataframe containing boolean columns y_test & y_pred
        k      -> integer number of items to consider
        y_test -> string name of column containing actual user input
        y-pred -> string name of column containing recommendation output
        
    Output:
        Floating-point number of precision value for k items
    """       
    # extract the k rows
    dfK = df.head(k)
    # compute number of recommended items @k
    denominator = dfK[y_pred].sum()
    # compute number of recommended items that are relevant @k
    numerator = dfK[dfK[y_pred] & dfK[y_test]].shape[0]
    # return result
    if denominator > 0:
        return numerator/denominator
    else:
        return None

@check_inputs
def recall_at_k(df: pd.DataFrame, k: int, y_test: str, y_pred: str) -> float:
    """
    Function to compute recall@k for an input boolean dataframe
    
    Inputs:
        df     -> pandas dataframe containing boolean columns y_test & y_pred
        k      -> integer number of items to consider
        y_test -> string name of column containing actual user input
        y-pred -> string name of column containing recommendation output
        
    Output:
        Floating-point number of recall value for k items
    """    
    # extract the k rows
    dfK = df.head(k)
    # compute number of all relevant items
    denominator = df[y_test].sum()
    # compute number of recommended items that are relevant @k
    numerator = dfK[dfK[y_pred] & dfK[y_test]].shape[0]
    # return result
    if denominator > 0:
        return numerator/denominator
    else:
        return None